# Product2vec: Grocery Item Embeddings

The original [Word2vec](https://arxiv.org/pdf/1301.3781.pdf) algorithm was released in 2013 and since then revolutionized the field of NLP. Simply put, word2vec learns high dimensional representations of words to convert one hot encoded features to more dense representations that can be used in algorithms.

It enables computers to more easily do math on this representation and along the way can demonstrate some interesting mathematical operations using cosine similarity, showing things like the vector for *Queen* being very similar to the vector of *King* minus the word *male* vector plus the *female* vector.

Products in a grocery store can be thought in a similar way to the bag of words approach used in text analytics, and there are some [interesting applications](https://arxiv.org/pdf/1810.08577.pdf) from NLP being ported over to the retail space.  

The same way documents can be thought of being composed of a series of words, baskets can be thought of as composed of a series of items purchased. There is a notable difference, in that the context of words matters alot. The immediately adjacent words have a much larger impact on the meaning of a word then words far away from it. However in a grocery basket it's reasonable to think that the order items are scanned across a checkout does not necessarily have a large relevance, but rather their overall presence in a basket. 

There are plenty of [high quality data explorations](https://www.kaggle.com/philippsp/exploratory-analysis-instacart) available, so I'll be skipping that task. All customers having had 3 previous orders, and the task is on predicting reorders, the products should be ordered often enough that extremely rare products isn't a huge thing to worry about.

In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile
from datetime import datetime
%matplotlib inline

C:\Users\Gage\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# append our repeated orders with our prior orders, to get the complete t-log.
orders = (pd.read_csv('data/order_products__train.csv')
          .append(pd.read_csv('data/order_products__prior.csv'))
         )

#we'll use product_name strings
products = pd.read_csv('data/products.csv')

In [3]:
relevant_cols = ['order_id','product_name']

#downsample while I test the code for faster iteration on syntax. run full dataset before commit.
sample_size = 1

baskets = (orders
           .merge(products,on='product_id',how='left')
           .sample(frac=sample_size)
          )[relevant_cols]

#memory management on my local computer
del([orders,products])

In [4]:
baskets.sort_values(['order_id']).head(20)

,order_id,product_name
5,1,Bag of Organic Bananas
2,1,Organic Celery Hearts
7,1,Organic Whole String Cheese
3,1,Cucumber Kirby
1,1,Organic 4% Milk Fat Whole Milk Cottage Cheese
0,1,Bulgarian Yogurt
6,1,Organic Hass Avocado
4,1,Lightly Smoked Sardines in Olive Oil
1384619,2,Garlic Powder
1384625,2,Classic Blend Cole Slaw


### Embedding Size

This will matter as we use gensim's word2vec implementation for this task. The learning task for word2vec is predicting the a missing word given a window of words around it using a single hidden layer.  Rather than caring about the quality of the prediction, the weights of the hidden layer are what represent the product embedding that we will use. The number of neurons in the hidden layer is a tunable parameter. Unfortunately, there isn't great guidance on select for this, but eyeballing the resulting embeddings can give guidance on quality of fit. Some people recommend using the 4th root of unique tokens in our corpus, which I'll try.

A tunable parameter for the algorithm is the context window, how many words around the target word to use for our prediction task. Given the lack of order, we will want to use the size of the largest basket, 145 for this parameter.

In [5]:
num_items = baskets.product_name.nunique()
embedding_size = np.floor(num_items**0.25).astype('int')
print('''Let's use vectors of length {n} for {tokens} products'''.format(n=embedding_size, tokens = num_items))

biggest_basket = np.max(baskets.groupby('order_id').product_name.nunique())
print('''The biggest basket (window in our algorithm) will be {}'''.format(biggest_basket))

Let's use vectors of length 14 for 49685 products
The biggest basket (window in our algorithm) will be 145


### Shaping our Data
The gensim implementation of word2vec expects each document to be a list. Traditionally, each document is a list of words. In this case, each basket is a list of products. We will use the product name, which will be more expensive in memory but will make interpretation easier.

In [6]:
df_of_basket_lists = (baskets
        .groupby('order_id')
        .apply(lambda baskets :
                baskets.product_name
                .tolist()
               )
       )

#memory management
del(baskets)

In [7]:
df_of_basket_lists.head()

order_id
1    [Cucumber Kirby, Organic Hass Avocado, Organic...
2    [Coconut Butter, All Natural No Stir Creamy Al...
3    [Lemons, Unsweetened Almondmilk, Unsweetened C...
4    [Energy Drink, Oats & Chocolate Chewy Bars, Or...
5    [Original Black Box Tablewater Cracker, Americ...
dtype: object

In [8]:
model = Word2Vec(df_of_basket_lists, size=embedding_size, window=biggest_basket)

In [9]:
def cosine_similarity(word_u,word_v,model):
    """
    Cosine similarity gets the similarity for two products and computes the similarity 
    between two embeddings in our word2vec model
        
    Arguments:
        u - numpy array of shape (n,)        
        v - numpy array of shape (n,)

    Returns:
        cosine similarity between words u & v
    """
    #get embeddings from gensim model
    u = model.wv[word_u]
    v = model.wv[word_v]

    #compute similarity
    dot = np.dot(u, v)
    norm_u = np.sqrt(np.sum(u * u))
    norm_v = np.sqrt(np.sum(v * v))
    cosine_similarity = dot / (norm_u * norm_v)
    
    return cosine_similarity

In [10]:
#a pair of similar identity items
cosine_similarity('Organic Whole Milk','Organic Reduced Fat Milk',model)

0.945737

In [11]:
# a pair of very different items
cosine_similarity('Bag of Organic Bananas','Party Tumblers',model)

-0.38942885

In [12]:
# a pair of similar items within a department
cosine_similarity('Bag of Organic Bananas','Limes',model)

0.39023426

## Next Steps
It's a bit handwavy to select a few cases where your embeddings work and call it a great job.  There is no real source of truth in this task however, product hierarchies like what department or aisle products are a part of are directionally accurate but not a base truth. 

Building the simple vector math to learn representations of features like *organic* are of interest. Showing that Organic apples - apples + steak equals organic beef would help demonstrate we are learning a real representation of product features.

Predicting item re-ordering as in the initial challenge, would likely be aided by the embedding of the item being reordered.

Implementing this in Keras rather than leveraging the gensim implementation would also be a worthwile exercise.